In [28]:
!pip install -q pyomo

In [29]:
import numpy as np
from pyomo.environ import *

In [30]:
data=np.loadtxt('Knapsack.txt')

In [31]:
data

array([[ 26., 550.],
       [ 91.,  84.],
       [ 72.,  83.],
       [ 90.,  43.],
       [ 46.,   4.],
       [ 55.,  44.],
       [  8.,   6.],
       [ 35.,  82.],
       [ 75.,  92.],
       [ 61.,  25.],
       [ 15.,  83.],
       [ 77.,  56.],
       [ 40.,  18.],
       [ 63.,  58.],
       [ 75.,  14.],
       [ 29.,  48.],
       [ 75.,  70.],
       [ 17.,  96.],
       [ 78.,  32.],
       [ 40.,  68.],
       [ 44.,  92.],
       [ 40.,  60.],
       [ 20.,  55.],
       [ 23.,  12.],
       [ 55.,  15.],
       [ 65.,  38.],
       [ 12.,  40.]])

In [32]:
model=ConcreteModel()

In [33]:
M=data.shape[0] -1
N=data.shape[1]

In [34]:
row_indices=np.arange(M)
col_indices=np.arange(N)

In [35]:
model.x=Var(row_indices,domain=Binary)

In [36]:
model.objective=Objective(expr=sum(model.x[i]*data[i+1][0] for i in row_indices),sense=maximize)

In [37]:
model.constraint=ConstraintList()

In [38]:
model.constraint.add(expr=sum(model.x[i]*data[i+1][1] for i in row_indices)<= data[0][1])

In [39]:
model.pprint()

2 Set Declarations
    constraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25}

1 Var Declarations
    x : Size=26, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Bina

In [40]:
!apt-get install -y -qq coinor-cbc

In [41]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Solver status: ok
Solver termination condition: optimal


In [42]:
# display solution
print('\nOptimal Solution: = ', model.objective())

print('\nDecision Variables')
for i in row_indices:
    if(model.x[i].value != 0):
      print('Item ',i+1,'is selected')


Optimal Solution: =  911.0

Decision Variables
Item  1 is selected
Item  2 is selected
Item  3 is selected
Item  4 is selected
Item  5 is selected
Item  6 is selected
Item  9 is selected
Item  11 is selected
Item  12 is selected
Item  14 is selected
Item  16 is selected
Item  18 is selected
Item  23 is selected
Item  24 is selected
Item  25 is selected


In [43]:
print(result.solver.time)

0.06099748611450195


In [44]:
from timeit import default_timer as timer

In [45]:
data=data.astype(int)

In [46]:
def knapsack_dp_with_items(items):
  n=int(data[0][0])
  W=int(data[0][1])
  dp = [[0 for _ in range(W+1)] for _ in range(n+1)]
  for i in range(1, n+1):
      for j in range(1, W+1):
          if items[i][1] > j:
              dp[i][j] = dp[i-1][j]
          else:
              dp[i][j] = max(dp[i-1][j], dp[i-1][j-items[i][1]] + items[i][0])
  included_items = []
  j = W
  for i in range(n, 0, -1):
      if dp[i][j] != dp[i-1][j]:
          included_items.append(i)
          j -= items[i][1]
      if j == 0:
          break
  included_items.reverse()
  return dp[n][W], included_items

In [47]:
start_time = timer()
max_value, included_items = knapsack_dp_with_items(data)
end_time = timer()
time_taken = end_time - start_time
print("Maximum value that can be achieved:", max_value)
print("Items included in the optimal solution:", included_items)
print("Time Taken: ", time_taken)

Maximum value that can be achieved: 911
Items included in the optimal solution: [1, 2, 3, 4, 5, 6, 9, 11, 12, 14, 16, 18, 23, 24, 25]
Time Taken:  0.048188738000135345


#1.5
Time taken by pyomo:0.06099748611450195\
Time taken by dynamic programming:0.048188738000135345\
The time to solve the model using dynamic programming was smaller than pyomo and it is quite less. So dynamic programming is better in terms of time taken